## Calculate the S/N ratio of all the spectra

In [127]:
import astropy.io.fits as fits
import matplotlib.pylab as plt
import numpy as np
from scipy.signal import medfilt
from scipy.interpolate import interp1d
from scipy.interpolate import splev, splrep
import math
from mpmath import mp
import os
import os.path 
import random
from random import seed
from random import choice
from random import uniform
import mplcursors
import pandas as pd
%matplotlib inline

## Read in all the files from Anna's directory
These are all the spectra that are used.

In [128]:
directory = '/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/NDR_corrected_wl_scale'

# every file in Anna's NDR_corrected_wl_scale folder
list_of_files = []
for filename in os.listdir(directory):
    if filename.endswith(".fits"): 
        list_of_files = np.append(list_of_files, filename)

Create a list of all the files to iterate through in the original APF data directory. Anna's repository only contains the reduced spectra. To calculate S/N ratios, we want to use the original APF data.

In [129]:
new_list_of_files = [s.replace(".NDR", "") for s in list_of_files]

In [130]:
original_files = ['/datag/blpd0/datax/apf/' + s for s in new_list_of_files]

Calculate SNR for all the stars in Anna's directory by going through the original data in the APF directory. SNR is estimated as the square root of the median of the flux value in the original APF data.

In [131]:
# wavelength solution
APF_wavelength_path = '/mnt_home/zoek/code/APF-BL-DAP/Zoe/APFTutorial/apf_wav.fits'
wl_file = fits.open(APF_wavelength_path)
wl = wl_file[0].data

In [132]:
all_SNR_vals = []

for file in original_files:

    file = fits.open(file)
    flux = file[0].data
    
    SNR_vals = []
    for order in np.arange(31, 52):  # use orders 31 to 52 because those are the orders included in Anna's reduced data
        median_fl = np.median(flux[order])
        if median_fl < 0:
            SNR = 0
        else:
            SNR = np.sqrt(median_fl)
        SNR_vals = np.append(SNR_vals, SNR)
    star_SNR = np.average(SNR_vals)
    
    all_SNR_vals = np.append(all_SNR_vals, star_SNR)
    

In [133]:
np.save('/mnt_home/zoek/code/APF-BL-DAP/Zoe/SNR.npy', all_SNR_vals)

Remove values at indicies 1762 and 1760 because those aren't included (took too long to run)
Also remove any item in the list that is 'nan'

In [85]:
new1 = np.delete(all_SNR_vals, 1762)

In [86]:
new2 = np.delete(new1, 1760)

In [87]:
final_SNR_vals = new2